# clio

In [1]:
from bs4 import BeautifulSoup as bs              # 데이터파싱 라이브러리
from selenium import webdriver
import json
import platform
import re
import time
import copy

In [2]:
def getNumber(string):
    numExtracter = re.compile('[0-9]+')
    return int(''.join(numExtracter.findall(string)))
    

In [3]:
def readNextPage():
    html = driver.page_source
    soup = bs(html, 'html.parser')
    pages = soup.find('div',{'class':'paginate'}).find_all('a')
    nextPage= False
    for page in pages:
        if nextPage:
            driver.get(url_products + page['href'])
            nextPage= False
        if page.find('strong'):
            nextPage= True
        
    

In [4]:
def getItemList():
    html = driver.page_source
    soup = bs(html, 'html.parser')
    items = soup.find('div',{'id':'pro_list'}).ul.find_all('li')
    itemList=[]
    for item in items:
        itemList.append('http://www.cliocosmetic.com/ko/product/' + item.a['href'])

    return itemList


In [5]:
def seeDetailInfo():
    html = driver.page_source
    soup = bs(html, 'html.parser')
    seeDetail = soup.find('button',text='구매하러가기')['onclick']
    driver.get(seeDetail[seeDetail.find('http://'):-1])
    

In [11]:
def getItem():
    html = driver.page_source
    soup = bs(html, 'html.parser')

    info = soup.find('div',{'class':'info_area'})

    brandName = 'clio'

    name = info.find('div',{'class':'tit'}).get_text().strip()
    if brandName in name: name = name[len(brandName)+2:].strip()

    image = soup.find('img',{'id':'overimage'})['src']

    category = '#'

    volume = soup.find('th',text='용량 또는 중량').parent.td.get_text().strip()
    if not volume: volume = '#'

    salePrice = getNumber(info.find('em',{'class':'point'}).get_text())

    originalPrice = info.find('del')
    originalPrice = getNumber(originalPrice.get_text()) if originalPrice else salePrice


    item={'name':'#', 'url':'#', 'image':'#', 'salePrice':'#', 'originalPrice':'#', 'color':'#', 
                       'category':'#', 'brand':'#','volume':'#'}
    item['name'] = name
    item['image']= image
    item['category']=category
    item['volume']=volume
    item['salePrice']=salePrice
    item['originalPrice']=originalPrice
    item['brand']=brandName
    item['url']=driver.current_url

    items=[]
    colorList = soup.find('select',{'id':'O_1'})
    if colorList:
        colors = colorList.find_all('option')
        for color in colors:
            color = color.get_text().strip()
            if '옵션' in color:
                continue
            if '색상' in color:
                continue
            if '선택' in color:
                continue

            if 'NEW' in color:
                color = color[4:].strip()
                
            item_copy = copy.deepcopy(item)
            item_copy['color'] = color
            items.append(item_copy)
    else:
        item['color']='#'
        items.append(item)
    display(items)
    return items

In [7]:
def writeJSON(jsonString, output_name='data.json'):
    with open(output_name,'w',encoding='UTF-8') as file:
        file.write(jsonString)


In [12]:
path = 'chromedriver.exe' if (platform.system() == 'Windows') else '/Users/jg/Desktop/develop/DataTeam/DataProcessing/product/crawling/chromedriver';
driver = webdriver.Chrome(path)

url_home = 'http://www.cliocosmetic.com/ko/product/list.asp'
url_products = 'http://www.cliocosmetic.com/ko/product/list.asp'

In [13]:
driver.get(url_products)

itemList = []
while True:
    try:
        itemList += getItemList()
        readNextPage()
    except:
        break
        

In [14]:
result=[]
for item in itemList:
    driver.get(item)
    seeDetailInfo()
    result += getItem()

[{'name': '[클리오] [어디서든 빛나는 오로라 광채피부]빅 오로라 광채쿠션',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201812267841',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/WM1370QQB9ZDJFEFNREX.jpg',
  'salePrice': 22400,
  'originalPrice': 32000,
  'color': '002 란제리',
  'category': '#',
  'brand': 'clio',
  'volume': '25g'},
 {'name': '[클리오] [어디서든 빛나는 오로라 광채피부]빅 오로라 광채쿠션',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201812267841',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/WM1370QQB9ZDJFEFNREX.jpg',
  'salePrice': 22400,
  'originalPrice': 32000,
  'color': '003 리넨',
  'category': '#',
  'brand': 'clio',
  'volume': '25g'},
 {'name': '[클리오] [어디서든 빛나는 오로라 광채피부]빅 오로라 광채쿠션',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201812267841',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/WM1370QQB9ZDJFEFNREX.jpg',
  'salePrice': 22400,
  'originalPrice': 32000,
  'colo

[{'name': '[클리오] [두드릴수록 살아나는 수분광채]하이드로 메이크업 스폰지 오리지널 마스터 블랜더',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201812113376',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/U1NX9KX0GI4AGKJQ7OB8.jpg',
  'salePrice': 6300,
  'originalPrice': 9000,
  'color': '#',
  'category': '#',
  'brand': 'clio',
  'volume': '1pcs'}]

[{'name': '[클리오] ★코랄 소량재입고!★[품절대란,영롱광채]프리즘 에어 섀도우 스파클링',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201811132695',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/L6GJJ2X1KX4ZCC5OEJ1Y.jpg',
  'salePrice': 9800,
  'originalPrice': 14000,
  'color': '016 코랄',
  'category': '#',
  'brand': 'clio',
  'volume': '2.3g'},
 {'name': '[클리오] ★코랄 소량재입고!★[품절대란,영롱광채]프리즘 에어 섀도우 스파클링',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201811132695',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/L6GJJ2X1KX4ZCC5OEJ1Y.jpg',
  'salePrice': 9800,
  'originalPrice': 14000,
  'color': '017 쿠퍼',
  'category': '#',
  'brand': 'clio',
  'volume': '2.3g'},
 {'name': '[클리오] ★코랄 소량재입고!★[품절대란,영롱광채]프리즘 에어 섀도우 스파클링',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201811132695',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/L6GJJ2X1KX4ZCC5OEJ1Y.jpg',
  'salePrice': 9800,
  'original

[{'name': '[클리오] ★18FW 홀리데이★[어드벤처 컬렉션] 프로 플레이 미니브러쉬 세트 [파우치+브러쉬5개구성]',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201811076799',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/G4977I985MTJEI39CAPG.jpg',
  'salePrice': 21000,
  'originalPrice': 30000,
  'color': '#',
  'category': '#',
  'brand': 'clio',
  'volume': '5EA'}]

[{'name': '[클리오] ★18FW 홀리데이★[어드벤처 컬렉션] 하이드로 메이크업 스폰지(하우스)',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201811078840',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/D1USCKLNAF4G5OP8H78L.jpg',
  'salePrice': 4200,
  'originalPrice': 6000,
  'color': '#',
  'category': '#',
  'brand': 'clio',
  'volume': '4pc'}]

...

[{'name': '[클리오] 컬링 업 뷰러',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201304114697',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/2(2).jpg',
  'salePrice': 2100,
  'originalPrice': 3000,
  'color': '#',
  'category': '#',
  'brand': 'clio',
  'volume': '1ea'}]

[{'name': '[클리오] 고급코튼면봉(200개)',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201304119960',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/4.jpg',
  'salePrice': 2100,
  'originalPrice': 3000,
  'color': '#',
  'category': '#',
  'brand': 'clio',
  'volume': ''}]

[{'name': '[클리오] 네일스타일러1+1',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201208245624',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/B3478CUB4W25UI2H06DH.jpg',
  'salePrice': 5000,
  'originalPrice': 5000,
  'color': '#',
  'category': '#',
  'brand': 'clio',
  'volume': '13ml'}]

[{'name': '[클리오] 네일 스타일러 리무버',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201208249573',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/NailSTremover.jpg',
  'salePrice': 2000,
  'originalPrice': 2000,
  'color': '#',
  'category': '#',
  'brand': 'clio',
  'volume': '100ml'}]

[{'name': '[클리오] 쿠션 퍼프(2매)',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201208232134',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/DCPVTFKUF0W0AKBHV2PY.jpg',
  'salePrice': 3500,
  'originalPrice': 5000,
  'color': '#',
  'category': '#',
  'brand': 'clio',
  'volume': '2P'}]

In [15]:
output = json.dumps(result,ensure_ascii=False, indent='\t')

writeJSON(output, output_name = 'clio.json')